In [1]:
from sys import path

path += ['..']

In [2]:
from mylib import Puller, PlotDisplayer, Slider
import asyncio
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

In [3]:
pl = Puller()

[(13, 90113195), (13, 90113196), (13, 90113197)]


C:\Users\ariva\projects\table_control\jupiter\..\mylib\motors.py:230: RuntimeWarning: coroutine 'Motor.MoveTo' was never awaited
  self.MoveTo(self.saveX, self.saveV, self.saveA)


In [4]:
async def lmot(x, v, a):
    for _ in range(3):
        await pl.ms.motorL.Move(x, v, a)
        await pl.ms.motorL.Move(-x, v, a)


async def rmot(x, v, a):
    for _ in range(3):
        await pl.ms.motorR.Move(x, v, a)
        await pl.ms.motorR.Move(-x, v, a)


async def mmot(x, v, a):
    for _ in range(10):
        await pl.ms.motorM.Move(x, v, a)
        await pl.ms.motorM.Move(-x, v, a)

In [5]:
async def run_all(x=10, v=10, a=5):
    tasks = []
    if type(x) == tuple:
        tasks.append(asyncio.create_task(lmot(x=x[0], v=v[0], a=a[0])))
        tasks.append(asyncio.create_task(rmot(x=x[1], v=v[1], a=a[1])))
        tasks.append(asyncio.create_task(mmot(x=x[2], v=v[2], a=a[2])))
    else:
        tasks.append(asyncio.create_task(lmot(x=x, v=v, a=a)))
        tasks.append(asyncio.create_task(rmot(x=x, v=v, a=a)))
        tasks.append(asyncio.create_task(mmot(x=x, v=v, a=a)))
    await asyncio.wait(tasks)

In [6]:
await run_all((10, 10, 1), (15,) * 3, (20,) * 3)

error in  motor L  -  position is bad x= 101 max position= 99


In [6]:
dp = PlotDisplayer()
# dp2=PlotDisplayer()
# dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
#     'time': pl.times,
#     'kof': pl.trueKmas
# }))
dp.CreateMaket(
    mainParam='time',
    power=pl.data[['time', 'power']],
    tension=pl.data[['time', 'tension', 'tensionWgl', 'tensionEXPgl']],
    motors=pl.data[['time', 'motorR', 'motorL', 'motorM']])
pl.pd = dp
sl = Slider()
sl.NewSl('v', min=0.1, max=9, value=5, step=0.1)
sl.NewSl('a', min=0.1, max=9, value=5, step=0.1)
sl.NewSl('window', min=50, max=10000, value=100, step=50)
sl.NewSl('molibdenH', min=-10, max=2.5, value=0.0, step=0.1)
sl.NewSl('Kp', min=0, max=10, value=0.15, step=0.01)
sl.NewSl('Ki', min=0, max=10, value=0.4, step=0.01)
sl.NewSl('Kd', min=-100, max=100, value=0, step=0.01)
sl.NewSl('T0', min=0, max=200, value=5 * 2, step=0.01)
sl.NewSl('end', min=0, max=1, step=1, value=0)
pl.sl = sl

In [7]:
await pl.ms.MoveToStart(
    zapas=10)  # сдвиг моторчиков на начальные позиции для растяжки

0

In [ ]:
pl.ms.x0 = 80
async def g():
    async with pl:
        pass


pl.sl.Display()
asyncio.create_task(g())

In [ ]:
await asyncio.wait([
    pl.ms.motorM.MoveTo(0),
    pl.ms.motorL.MoveTo(0),
    pl.ms.motorR.MoveTo(0)
])